<a href="https://colab.research.google.com/github/KimKangYeon/HW_/blob/master/%EC%A4%91%EA%B0%84%EA%B3%BC%EC%A0%9C_2%EB%B2%88%EC%A7%B8_%ED%8C%8C%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
# 텐서플로어 라이브러리를 임포트해주었습니다.

     |████████████████████████████████| 380.5MB 47kB/s 
     |████████████████████████████████| 501kB 60.5MB/s 
     |████████████████████████████████| 4.3MB 49.1MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 


In [2]:
# 그런 다음 mnist dataset을 로드해주었습니다.
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# multiple dimention을 추가할 때와 slice를 선택하고 새로운 dimensions을 추가할 때 편리함을 위해 train 변수와 test 변수에 a[:,:,tf.newaxis]:을 이용하여 차원을 추가해주었습니다.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(5000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)
# 많은 양의 데이터를 처리해야하기 때문에 tf.data api를 이용하여 복잡한 변환을 수행하였습니다. 이 과정에서 dataset을 셔플해주고 배치를 만들어주었습니다. 
# 다른 파라미터는 고정시키고 셔플 횟수와 배치 개수를 변경해보며 경향성을 파악한 후 가장 높은 정확도를 가지는 값을 찾아보았습니다. 


In [4]:
# 그런다음 model subclassing api를 사용해서 모델을 만들었습니다.
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(256, activation='relu')
    self.d2 = Dense(128, activation='relu')
    # self.d3 = tf.keras.layers.Dropout(0.2)
    self.d3 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d2(x)    
    return self.d3(x)

model = MyModel()
# relu activation function은 Monotonic하고(단조롭고 즉, 증가만 하거나 감소만 하고) Derivative Monotonic 하기 때문에 
# 어떤 입력 값이 커질수록 어떤 경향을 보인다로 해석 가능하고 어떤 입력 값의 기울기는 어떤 경향을 보인다로 해석 가능하다는 이점이 있어 사용하였습니다.
# 또한 위키백과를 통해 "Better gradient propagation: Fewer vanishing gradient problems compared to sigmoidal activation functions that saturate in both directions"하다는
# 사실을 알게 되어 조교님께서 예제 코드에서 사용하셨던 sigmoid 함수 대신 relu를 사용하였습니다.
# 그리고 위키백과를 통해 추가적으로 "Biological plausibility: One-sided, compared to the antisymmetry of tanh"하다는 점을 이용하여 tanh 대신 relu를 사용하였습니다.
# 입력과 출력을 모두 연결해주는 역할을 하는 Dense 레이어를 사용하였습니다. 예를 들어 입력 뉴런이 n개, 출력 뉴런이 m개있다면 총 연결선은 n*m개입니다. 
# 각 연결선은 가중치(weight)를 포함하고 있고 그 가중치가 높을수록 해당 입력 뉴런이 출력 뉴런에 미치는 영향이 크고, 낮을수록 미치는 영향이 적다는 점을 이용하여 적절한 출력뉴런의 수를 설정해주었습니다.
# Dense(출력 뉴런의 수, 입력 뉴런의 수, 가중치 초기화 방법, activation='활성화 함수 설정') 구조를 이용하여 해당 값들을 바꿔가며 가장 높은 정확도를 가지는 값을 찾아보았습니다.
# 소프트맥스 함수는 다중 클래스 분류에서 출력층에 주로 사용하기 때문에 마지막 출력층에 넣었습니다.
# Dense 레이어는 입력 뉴런 수에 상관없이 출력 뉴런 수를 자유롭게 설정할 수 있기 때문에 출력층으로 사용하였습니다. 
# 입력 뉴런과 가중치를 계산한 값을 0에서 1사이로 표현할 수 있는 활성화 함수 중에서 앞서 언급한 이유로 인해 sigmoid 함수 대신 relu 함수를 사용하였습니다.
# 아래의 텍스트로 파라미터들을 변화시켜가며 최고의 정확도를 보이는 값을 찾아가는 과정을 첨부하였습니다. 

셔플횟수 /  	batch size /	트레이닝정확도(%)	/ 테스트정확도(%)	/ 에포크 수		



---



10000	/ 32 /	98.59	 / 98.12999	/ 5		   (초기 상태)

5000 /	32 /	98.67	 / 98.3199 /	5		  (셔플 횟수 변화시켰을 때)

5000 /	64 /	98.47 /	98.202 /	5		     (batch size 변화시켰을 때)
						
5000 /	32 /	99.2245 /	98.191 /	10		(epoch 변화시켰을 때)
						
5000 /	32 /	98.67 /	98.3199 /	5     	(Dense 레이어 하나만 있을 때( 출력뉴런의 수는 256))	       **[두번째로 높은 정확도]**

5000 /	32 /	98.67 /	98.342 /	5	  (레이어를 하나 추가하였을 때( 출력뉴런의 수는 256, 128))	**[가장 높은 정확도]**

5000 /	32 /	98.36 /	98.18 /	5	  (레이어를 두개 추가하였을 때( 출력뉴런의 수는 256, 128,  64))	



---

셔플 횟수와 batch size, epoch 수와 레이어를 변화시켜가며 가장 높은 정확도를 얻을 때의 값을 알아보았습니다. 

이제 다른 파라미터는 고정시킨 다음 regularization으로 Dropout(rate)을 주었을 때

 정확도에 어떤 영향을 미치는지 알아보겠습니다.

regularization으로 Dropout(rate = 0.1)을 주었을 때 (출력뉴런의 수가 256인 Dense 레이어가 하나만 있을때)

트레이닝정확도(%)	/ 테스트정확도(%)

98.76  / 98.22

regularization으로 Dropout(rate = 0.1)을 주었을 때 (출력뉴런의 수가 256, 128인 Dense 레이어가 두 개 있을때)

98.63 / 98.28

regularization으로 Dropout(rate = 0.2)을 주었을 때 (출력뉴런의 수가 256, 128인 Dense 레이어가 두 개 있을때)

98.68 / 98.25

따라서 이 결과를 통해 regularization으로 Dropout을 주었을 때 정확도가 오히려 감소한다는 것을 알 수 있었습니다. 


In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

# 트레이닝에 필요한 optimizer와 loss function의 종류는 각각 Adam과 crossentropy loss function을 사용하였습니다.
# 많은 옵티마이저 중 keras.optimizers.Adam(Adaptive moment estimation)은 기존 RMSprop과 momentum과 다르게 바이어스가 수정된 값으로 변경하는 과정이 포함되어있기 때문에 선택해보았습니다.
# 또한 Adam의 경우 α=0.001, β1으로는 0.9, β2로는 0.999, ϵ 으로는 10^-8 값이 가장 좋은 Default값이라고 논문에 명시되어 있고 이 값이 이미 빌트인 되어 있기 때문에 lr(learning rate)을 변경하지 않았습니다.
# crossentropy loss function는 두 개 이상의 label class가 있을 때 유용하기 때문에 선택해보았습니다.

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
# model의 loss와 accuracy을 측정할 지표를 선택하고 epoch가 진행되는 동안 수집된 측정 지표를 바탕으로 최종 결과를 출력하도록 하였습니다.

In [7]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
  # 자동 미분(주어진 입력 변수에 대한 연산의 그래디언트(gradient)를 계산하는 것)을 위해 tf.GradientTape ap를 사용하였습니다. 
  # tf.GradientTape는 context 안에서 실행된 모든 연산을 tape에 기록한 다음 후진 방식 자동 미분(reverse mode differentiation)을 사용해 테이프에 기록된 연산의 그래디언트를 계산하게됩니다.
  # tf.GradientTape를 이용하여 model을 training하였습니다.

In [8]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)
  # 이 부분은 model을 테스트하기 위한 함수를 정의해주었습니다.

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))
# epoch는 다른 파라미터들은 고정 시킨 뒤 epoch 값만 변화시켜 10일 때와 5일 때를 비교하였을 때 5일 때가 더 높은 훈련정확도와 테스트정확도를 보여 5로 설정하였습니다.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

에포크: 1, 손실: 0.1300235539674759, 정확도: 96.01499938964844, 테스트 손실: 0.07052401453256607, 테스트 정확도: 97.8499984741211
에포크: 2, 손실: 0.08607939630746841, 정확도: 97.36499786376953, 테스트 손실: 0.06272236257791519, 테스트 정확도: 98.04499816894531
에포크: 3, 손실: 0.06479645520448685, 정확도: 98.00389099121094, 테스트 손실: 0.0629410520195961, 테스트 정확도: 98.11333465576172
에포크: 4, 손실: 0.05268470197916031, 정확도: 98.35958862304688, 테스트 손실: 0.06400776654481888, 테스트 정확도: 98.11750030517578
에포크: 5, 손실: 0.04426215589046478, 정확도: 98.61866760253906, 테스

In [10]:
!mkdir -p saved_model
model.save('saved_model/my_model')


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets
